<p style="float:right">
<img src="images/logos/cu.png" style="display:inline" />
<img src="images/logos/cires.png" style="display:inline" />
<img src="images/logos/nasa.png" style="display:inline" />
<img src="images/logos/nsidc_daac.png" style="display:inline" />
</p>

# Python, Jupyter & pandas: Module 2

## Obtaining data and basic inspection

### Basic data access

It is, of course, possible to obtain data (roughly construed -- we'll look at images here because they're simple to view) outside Jupyter -- or via one of the script (e.g. `%%bash`) magics, which saves the trouble of opening a separate terminal / command / browser window).

Here we fetch an image to the local filesystem, then display it with Markdown (you may need to double-click the Markdown cell below the code and re-evalute it with Shift+Enter or the "play" button in the toolbar):

In [ ]:
%%bash
imgfile='N_197902_extn.png'
rm -f $imgfile
wget ftp://sidads.colorado.edu/DATASETS/NOAA/G02135/Feb/$imgfile

<img src='N_197902_extn.png' style='float:left'/>

We can also obtain an image directly from the Internet and display in with Python code:

In [ ]:
from IPython.display import Image
Image(url='ftp://sidads.colorado.edu/DATASETS/NOAA/G02135/Feb/N_201602_extn.png')

The `netCDF4` package provide OPeNDAP client capabilities. Here we use it to obtain data via an OPeNDAP server at NSIDC:

In [ ]:
import netCDF4
url = ('http://opendap.apps.nsidc.org:80/opendap/DATASETS/'
       'nsidc0530_MEASURES_nhsnow_daily25/2012/nhtsd25e2_20120101_v01r01.nc'
)
dataset = netCDF4.Dataset(url)

### Data inspection

We can inspect the `dataset` object to see its class. In this case, it's exactly what we'd expect given that we created it with `netCDF4.Dataset`. However, it's sometimes the case, especially when working with a new library, that we do not anticipate the type of an object returned from some method / function call, so it's handy to be able to find out what it is.

In [ ]:
type(dataset)

Given that we have a `Dataset` object from the `netCDF4` library, we could of course go consult [that library's documentation](https://netcdf4-python.googlecode.com/svn/trunk/docs/netCDF4.Dataset-class.html) to learn what kinds of attributes and methods such an object has.

Or, we can bravely plunge in and have a look for ourselves.

The most general kind of inspection we can do is to simply ask an object to express itself as a string:

In [ ]:
dataset

In this case, we get lots of useful information. Not all Python objects are so helpful.

Well-written Python code includes so-called _docstrings_, which we can access via the built-in `help` command:

In [ ]:
help(dataset)

In this case, we get extensive -- and quite helpful -- information on the `Dataset` object.

We can also use Python's built-in `dir` command to get a list of object members. This mechanism can be helpful even in cases where docstrings are unavailable.

In [ ]:
dir(dataset)

Unfortunately, this list includes attributes and/or methods, not intended for end-users like us. The convention in Python is to name "private" attributes/methods with one or more leading underscores. Here, we filter out those members:

In [ ]:
[x for x in dir(dataset) if not x.startswith('_')]

This is getting closer, but there's still a lot here that may not be of interest to us. Since this is NetCDF data, though, we can call the `ncattrs` method on our `dataset` variable to get an even more specific list of attributes specific to this data.

In [ ]:
dataset.ncattrs()

We can inspect these attributes to see their values, which match information on [this dataset's documentation page](http://nsidc.org/data/docs/measures/nsidc-0530/index.html) at NSIDC.

In [ ]:
dataset.title

In [ ]:
dataset.institution

The actual data is available under `variables`: 

In [ ]:
for variable in dataset.variables:
    print(variable)

Note that these variables correspond to those listed in Table 3 of the documentation page linked to above.

Let's extract the `latitude` variable and look at its properties.

In [ ]:
latitude = dataset.variables['latitude']
latitude

So, `latitude` is a 720 x 720 array, with valid values ranging from -90 to 90 degrees north, and invalid ("fill") marked as -999.

Since we pulled `latitude` out of another object, rather than creating it explicitly as we did with the `Dataset`, what kind of object do we have?

In [ ]:
type(latitude)

Makes sense. And how are the data in this variable represented?

In [ ]:
latitude.datatype

As 32-bit floating-point numbers. We also saw, but might not have noticed, this when we printed `latitude`, above: Note the `float32` designation in the second line. Similarly, all the other data shown above can be extracted with more targeted queries:

In [ ]:
latitude.long_name

In [ ]:
latitude.valid_range

In [ ]:
latitude.shape

Just as with our `Dataset`, we can look at all the public attributes and methods of our `Variable`:

In [ ]:
[x for x in dir(latitude) if not x.startswith('_')]

Instances of the `Variable` class (like our `latitude` object) from `netCDF4` behave like multidimensional arrays, similar to NumPy's `ndarray`. So, we can access elements with the familiar `[]` bracket notation. Since we know that `latitude` is 720 x 720, as we expect:

In [ ]:
len(latitude)

And if we look at the first row in the `latitude` array, its length is similarly what we'd expect:

In [ ]:
len(latitude[0])

Note that Python is zero-indexed like C, and unlike Fortran, so valid indices range from 0 to 719.

Just like the `Dataset`, our `netCDF4.Variable` has an `ncattrs()` method that will describe the NetCDF Variable Attributes:

In [ ]:
for attribute in latitude.ncattrs():
    print(attribute,' => ', latitude.getncattr(attribute))

Let's extract the `time` variable from our dataset and examine it. (Note that we can view output from commands other than the last one in a cell by explicitly using Python's `print` command.)

In [ ]:
time = dataset.variables['time']
print(time)
print(time[0])

So, this dataset's data starts 4749 days after 1998-12-31 on the Gregorian calendar. Let's convert that information into something more useful:

In [ ]:
units = time.units
print('units: %s' % units)
value = time[0]
print('value: %s' % value)
print(netCDF4.num2date([value], units)[0])
netCDF4.num2date([value], units)[0].strftime('%Y-%m-%d')

Happily, this agrees with the URL we used to request this data in the first place. If we had a directory full of data files to process, we could obtain this information internally rather than relying on metadata encoded in filenames.

Out of curiosity, let's check that the `longitude` variable's shape conforms to that of `latitude`, as we'd hope.

In [ ]:
longitude = dataset.variables['longitude']
longitude

Good, it is 720 x 720 just like `latitude`.

Now let's look at one of the actual snow cover variables which, presumably, is why we're bothering with this dataset in the first place:

In [ ]:
msce = dataset.variables['merged_snow_cover_extent']
msce

Inspecting the Merged Snow Cover Extent variable, we see that the data is in a 720 x 720 array (the dimensions match those of latitude and longitude, so that's good!) whose values are integers specifying snow cover information from various sources, as well as snow-free and ice-covered land, and ocean.

A "fill" value of -99 is used to indicate bad or unavailable data.

We can pick a "random" array element and see its value:

In [ ]:
msce[0][360][360]

40 = Ocean. Does it make sense?

In [ ]:
print(latitude[360][360])
print(longitude[360][360])

That's pretty close to the north pole, in the Arctic Ocean, so seems reasonable.

Let's do a quick-and-dirty visualization of this data as another reality check:

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
plt.imshow(msce[0, :, :])

Looking down on the North Pole, we can recognize familiar geographic shapes: North America to the left, Asia to the right, and Greenland in yellow near the center. Note that the scale of the axes are in rows and columns, not latitudes and longitudes. Still, this is a helpful visual verification that we are looking at reasonable data.

Let's use NumPy to convert our `msce` variable into an `ndarray` object, and get rid of that useless first dimension. NumPy's `squeeze` method removes all trivial (one-element) dimensions from a multidimensional array.

In [ ]:
import numpy as np
msce = np.squeeze(msce)
msce.shape

That's better. Now `msce`'s shape matches that of `latitude` and `longitude`.

How much _good_ data do we have in `msce`? That is, how many data elements are there in total vs those set to the fill value?

In [ ]:
print(msce.size)
print(msce[msce != -99].size)

So, over 20% of the data elements are set to the fill value. This sometimes happens with satellite (and other data): Quality Control (QC) algorithms determine that some observations are suspect, so they are marked as such so that further analysis can avoid depending on them.

In this case, though, and comparing to our simple visualization of the data, above, we can see that the "bad" values simply lie in positions of the 720 x 720 grid that do not fall on the Earth.

How many grid points match these criteria?

In [ ]:
plt.imshow(msce == -99, cmap="Blues")

### Subsetting data with OPeNDAP

One benefit to using OPeNDAP for data access is that data can be subsetted prior to download, to avoid the transfer and storage of data one is not interested in.

NSIDC's [OPeNDAP Server Dataset Access Form](http://opendap.apps.nsidc.org/opendap/DATASETS/nsidc0530_MEASURES_nhsnow_daily25/2012/nhtsd25e2_20120101_v01r01.nc.html) for this data gives some guidance on subsetting the data. For starters, let's restrict our query to the three variables -- Latitude, Longitude, and Merged Snow Cover Extent -- that we are interested in. When we tick the checkboxes for _latitude_, _longitude_, and *merged_snow_cover_extent*, the URL shown in the _Data URL_ field is updated to:

`http://opendap.apps.nsidc.org:80/opendap/DATASETS/nsidc0530_MEASURES_nhsnow_daily25/2012/nhtsd25e2_20120101_v01r01.nc?latitude[0:1:719][0:1:719],longitude[0:1:719][0:1:719],merged_snow_cover_extent[0:1:0][0:1:719][0:1:719]`

Let's perform our query and data again with this URL and check the variables we have now:

And how do they look?

In [ ]:
url = ('http://opendap.apps.nsidc.org:80/opendap/DATASETS/'
       'nsidc0530_MEASURES_nhsnow_daily25/2012/nhtsd25e2_20120101_v01r01.nc?'
       'latitude[0:1:719][0:1:719],'
       'longitude[0:1:719][0:1:719],'
       'merged_snow_cover_extent[0:1:0][0:1:719][0:1:719]'
)
dataset = netCDF4.Dataset(url)
for variable in dataset.variables:
    print(variable)

Previously, we had ten variables; now we have only three. Nice.

Let's say we're only interested in snow cover in Iceland. Let's subset the data geographically as well. The [OPeNDAP Server Dataset Access Form](http://opendap.apps.nsidc.org/opendap/DATASETS/nsidc0530_MEASURES_nhsnow_daily25/2012/nhtsd25e2_20120101_v01r01.nc.html) gives us options for constraining the variables, but expects us to do so by row and column. Iceland lies between 12 to 25 degrees west, and 63 to 67 degrees north. Let's find out which rows and columns in our Dataset fall within those bounds

We'll extract the data from our NetCDF `latitude` and `longitude` variables, then construct a boolean mask corresponding to values that surround Iceland. Since `latitude` and `longitude` have matching 720 x 720 shapes, our mask gets the same shape, and is composed of elements where **all** the conditions hold true. Note the use of the bitwise-and `&` operator.

In [ ]:
lat = latitude[:]
print(type(latitude))
print(type(lat))
lon = longitude[:]
in_range = (lat >= 63) & (lat <= 67) & (lon >= -25) & (lon <= -12)
in_range.shape

In [ ]:
np.sum(in_range)

In [ ]:
plt.imshow(in_range)

Let's use NumPy's [`where()`](http://docs.scipy.org/doc/numpy-1.10.1/reference/generated/numpy.where.html) function get row and column locations where `in_range` is true. This filters out all locations where the `in_range` conditions are `False`, so we should be able to sample `in_range` at selected `row` and `col` points and verify that we only see `True` values.

In [ ]:
row, col = np.where(in_range)
print(in_range[row[0:5], col[0:5]])

Given that `row` and `col` now represent only those rows and columns containing our area of Icelandic interest, we can compute a bounding box from the row/column minima and maxima:

In [ ]:
bbox = np.min(row), np.max(row), np.min(col), np.max(col)
print(bbox)

Now let's use our bounding-box values as constraints in our OPeNDAP URL to select just the rows and columns that we think correspond to Iceland. The OPeNDAP constraints are given in `lower_bound:stride:upper_bound` form:

In [ ]:
url = ('http://opendap.apps.nsidc.org:80/opendap/DATASETS/'
       'nsidc0530_MEASURES_nhsnow_daily25/2012/nhtsd25e2_20120101_v01r01.nc?'
       'latitude[453:1:476][310:1:338],'
       'longitude[453:1:476][310:1:338],'
       'merged_snow_cover_extent[0:1:0][453:1:476][310:1:338]'
)
dataset = netCDF4.Dataset(url)
dataset

Note the line `dimensions(sizes): time(1), cols(29), rows(24)`: 24 rows and 29 columns, which corresponds to our request. This is a lot less data than we were getting with the full 720 x 720 grids!

In Module 3, we'll display this geolocated data and see if we really got what we asked for.